In [16]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import pandas as pd
import torch
import os
import glob
import wandb
import ultralytics
import cv2
print('ultralytics %s' % ultralytics.__version__)
print('wandb %s' % wandb.__version__)
from wandb.integration.ultralytics import add_wandb_callback

ultralytics 8.3.75
wandb 0.19.6


In [17]:
!nvidia-smi

Fri Feb 14 12:02:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   43C    P0             29W /  125W |    1253MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Arguments setting for the model

In [18]:
args = dict(
    data = "data.yaml",
    epochs = 1,
    patience = 5,
    batch = 4,
    save = False,
    save_period = 5,
    device = '0',
    workers = 1,
    project = "weights",
    name = "try",
    exist_ok = False, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    conf = 0.01,
    iou = 0.6,
    degrees = 0.0, 
    shear = 0.0,
    perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

Loading the model yoloV8 and running wandb for logging

Training of model

In [20]:
settings.update({"wandb": True})
wandb.login()

True

In [21]:
wandb.init(project='YOLOv8-demo', name='test_run')

In [22]:
# for loading model
model = YOLO('yolov8n.pt')

In [23]:
add_wandb_callback(model, enable_model_checkpointing=True)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [24]:
#training model
results = model.train(cfg="train_cfg.yaml")


Ultralytics 8.3.75  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=4, time=None, patience=5, batch=4, imgsz=640, save=False, save_period=5, cache=False, device=0, workers=1, project=weights, name=try7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.6, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]
val: Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]


Plotting labels to weights\try7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to weights\try7
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/4      1.21G      1.455      2.359      1.199         10        640: 100%|██████████| 1185/1185 [03:02<00:00,  6.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:24<00:00, 15.72it/s]

                   all      10665       9969      0.815      0.738      0.799      0.542
Ultralytics 8.3.75  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


...


4 epochs completed in 0.243 hours.
Optimizer stripped from weights\try7\weights\last.pt, 6.2MB
Optimizer stripped from weights\try7\weights\best.pt, 6.2MB

Validating weights\try7\weights\best.pt...
Ultralytics 8.3.75  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:10<00:00, 18.88it/s]


                   all      10665       9969      0.878      0.847      0.878      0.639
               Glottis       9969       9969      0.878      0.847      0.878      0.639


Generating Visualizations for batch-1/1: 100%|██████████| 8/8 [00:00<00:00, 25.18it/s]

Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to weights\try7


In [27]:
model_path = 'yolov8nDemo.pt'
model.save(model_path)

In [28]:
wandb.log_model(model_path)
# wandb.finish()

UsageError: Run (qisv4zep) is finished. The call to `log_model` will be ignored. Please make sure that you are using an active run.

Running Predictions on model using bagls video and dhs video

Can use OpenCV for better video processing and visualization of the predictions

In [11]:
dhs_video = r'D:\FYP Code\testfiles\dhs.mp4'
baglsC_video = r'D:\FYP Code\testfiles\bagls-color.mp4'
bagls_video = r'D:\FYP Code\testfiles\bagls.mp4'
test_path = r'D:\FYP Datasets\fyp\training\test\images'
weight_path = r'weights\try\weights\best.pt'

In [12]:
model = YOLO(weight_path)
# model.predict(source= test_path, save_dir = 'results', save= True)
model.predict(source=dhs_video, save_dir = 'results', save= True)
model.predict(source=baglsC_video, save_dir = 'results', save= True)
model.predict(source=bagls_video, save_dir = 'results', save= True)




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.2ms
video 1/1 (frame 2/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 9.2ms
video 1/1 (frame 3/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.1ms
video 1/1 (frame 4/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 7.9ms
video 1/1 (frame 5/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.0ms
video 1/1 (frame 6/808) D:\F

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Torus Tobarius', 1: 'Soft palate', 2: 'Uvula', 3: 'Glottis', 4: 'True vocal folds', 5: 'Epiglottis'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [2, 2, 2],
         [2, 2, 2]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
     

In [13]:
# wandb.log({"final_mAP": results.metrics.mAP50})
# wandb.finish()